### Домашнее задание "Проблема качества данных"
Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. По образу практики, попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import random

from sklearn.linear_model import LinearRegression

In [17]:
# Создадим рандомные данные по стоимости квартиры в зависимости от возраста владельца и параметров квартиры
n_samples = 100000

length = np.random.choice(15, n_samples) + 5  # длина 
width = np.random.choice(10, n_samples) + 5  # ширина
flat = np.random.choice(25, n_samples) + 1  # Этаж

price = (length * width) * (1 + flat/100) * 250000
data = pd.DataFrame({'length': length, 'width': width, 'm2': length * width, 
                     #предполагаем, что чем выше этаж, тем дороже
                     'flat': flat, 'coef_flat':(1 + flat/100), 
                     'price': price})
data.head(5)

,length,width,m2,flat,coef_flat,price
0,19,6,114,8,1.08,30780000.0
1,15,5,75,12,1.12,21000000.0
2,19,13,247,4,1.04,64220000.0
3,10,6,60,19,1.19,17850000.0
4,11,9,99,25,1.25,30937500.0


In [18]:
data.describe()

,length,width,m2,flat,coef_flat,price
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05
mean,11.991220,9.501370,113.985250,13.019370,1.130194,3.220959e+07
std,4.326311,2.866273,55.118103,7.213619,0.072136,1.575930e+07
min,5.000000,5.000000,25.000000,1.000000,1.010000,6.312500e+06
25%,8.000000,7.000000,70.000000,7.000000,1.070000,1.980000e+07
50%,12.000000,9.000000,104.000000,13.000000,1.130000,2.921250e+07
75%,16.000000,12.000000,152.000000,19.000000,1.190000,4.248000e+07
max,19.000000,14.000000,266.000000,25.000000,1.250000,8.312500e+07


In [19]:
# предположим, что мы не знаем зависимость между параметрами
# cоздаём модель и обучаем её на наших признаках.
from sklearn.metrics import mean_absolute_error

X = data[['length', 'width', 'flat']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['length', 'width', 'flat']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [2684680.63049608 3395181.50549589  287064.39199242]
Bias: -35979280.32906493
Error: 2710542.852303532


In [20]:
y.median()

29212500.0

Наблюдаем очень большую ошибку предсказания (средняя абсолютная ошибка порядка 2710542 и bias -35979280).

Используем признак m2 

In [23]:
X = data[['m2']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['m2']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [282893.38215389]
Bias: -36083.91315701604
Error: 1780979.4838090318


Используем признак coef_flat

In [24]:
X = data[['coef_flat']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['coef_flat']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [29239472.30367356]
Bias: -836678.4139363468
Error: 12814652.59922529


Используя признак m2 ошибка предсказания существенно изменилась, в сторону улучшения  (средняя абсолютная ошибка порядка 1780979 и смещение bias -36083).  
Таким образом, можно сделать вывод, что предобработка позволяет улучшить результаты предсказания.